In [112]:
import numpy as np

In [113]:
datos = np.array([["audi", "gas", "std", 1],
                 ["audi", "gas", "std", 1],
                 ["audi", "gas", "std", 0],
                 ["audi", "gas", "std", 2],
                 ["mazda", "gas", "std", 1],
                 ["mazda", "gas", "std", 1],
                 ["mazda", "gas", "std", 2],
                 ["peugot", "diesel", "rwd", 2],
                 ["peugot", "diesel", "turbo", 2],
                 ["volvo", "gas", "turbo", 0],
                 ["volvo", "gas", "std", 1],
                 ["volvo", "gas", "turbo", 0]])

In [114]:
def return_unique_characteristics(column: int) -> list:
    characteristics = datos[:,column].tolist()
    characteristics

    unique_characteristics = []

    for characteristic in characteristics:
        if not(characteristic in unique_characteristics):
            unique_characteristics.append(characteristic)
    return unique_characteristics

def create_dictionary(unique_characteristics: list, total_class: int) -> dict:
    dictionary = dict()

    for characteristic in unique_characteristics:    
        for i in np.arange(total_class):
            key = characteristic + str(i)
            dictionary.setdefault(key, 0)
    return dictionary

In [117]:
total_sum = 0
for characteristic in unique_characteristics:
    simple_sum = 0
    ni = 0
    for i in np.arange(total_class):
        key = characteristic + str(i)
        simple_sum += dictionary[key] / datos.shape[0]
        ni += dictionary[key]
    print(simple_sum)
    simple_sum = 1 - simple_sum
    total_sum += (simple_sum * ni) / datos.shape[0] 
print(total_sum)

0.0
0.0
0.0
0.0
0.0
